### **imports**

In [25]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

### **making a table of all of the stars in the txt file**

In [26]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [27]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [28]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [29]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [30]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [41]:
num_iterations = 1
empty_list = []
for i in range(140, 150):
    if i == 8 or i == 121 or i == 141 or i == 148:
        continue
    search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    quality = g_filter[g_filter['quality'] == 'G']
    web_format = quality[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 1 (0 - classification success, 1 - suspicious period)
class = EB_ED
class_probability = 0.44


######### Lightcurve features #########

amplitude = 0.1231113444504893
cusum = 0.5949773900243829
eta = 4.636792954813404
hl_amp_ratio = 3.631621322063064
kurtosis = 1.9249414326615026
n_points = 799
period = 0.49933048980115363
period_SNR = 84.48081571155676
period_log10FAP = -38.86762250914662
period_uncertainty = 0.00010789336929867277
phase_cusum = 0.6418220831414452
phase_eta = 3.814985425144154
phi21 = -2.6750102932414683
phi31 = -5.018557111154495
quartile31 = 0.14703647996268465
r21 = 0.2646708057311711
r31 = 0.08222947568573075
shapiro_w = 0.7751968502998352
skewness = 1.6437095263321895
slope_per10 = -0.061078098839870046
slope_per90 = 0.04593324529217279
stetson_k = 0.835660411079065
weighted_mean = 11.583683116855237
weighted_std = 0.10904863140878433
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.3513272366144769
cusum = 0.18992974053892528
eta = 1.4211393190405108
hl_amp_ratio = 0.44425191734824837
kurtosis = 0.446226236500018
n_points = 1959
period = 0.5699101116513362
period_SNR = 213.1120948210273
period_log10FAP = -259.9627882766078
period_uncertainty = 0.00014018780242741613
phase_cusum = 0.3470236145184959
phase_eta = 0.3785782047228861
phi21 = -2.3988369963020384
phi31 = -4.468927625070965
quartile31 = 0.42552155234921507
r21 = 0.43486121710274694
r31 = 0.23131353174376207
shapiro_w = 0.9467988014221191
skewness = -0.4329829474426836
slope_per10 = -0.017180229078612174
slope_per90 = 0.016818883924075583
stetson_k = 0.7087397518023486
weighted_mean = 14.94728290635055
weighted_std = 0.2938476887068847
Pulled 1 of 1


...

/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.417684666918026
cusum = 0.24474230667448568
eta = 2.0552045243891945
hl_amp_ratio = 0.3629853734685813
kurtosis = -0.5946381828673979
n_points = 830
period = 0.6612652156046563
period_SNR = 191.70962033024273
period_log10FAP = -113.91218835906204
period_uncertainty = 0.0001892212713898167
phase_cusum = 0.39513628705289383
phase_eta = 0.1408198906079646
phi21 = 0.722245608689208
phi31 = 1.7974506951442368
quartile31 = 0.5290092263600563
r21 = 0.497822329201631
r31 = 0.30303201334880153
shapiro_w = 0.9222419261932373
skewness = -0.670405984297409
slope_per10 = -0.07107229421226163
slope_per90 = 0.08222671182691758
stetson_k = 0.6729422038750612
weighted_mean = 14.89110022259476
weighted_std = 0.3516761730119924
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_c
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.24784290146694568
cusum = 0.17021576035406893
eta = 2.3893712982610866
hl_amp_ratio = 0.946448932357675
kurtosis = -1.0818940133158437
n_points = 1033
period = 0.3007672595717243
period_SNR = 225.17328121509058
period_log10FAP = -176.51801116609374
period_uncertainty = 3.8877237011125665e-05
phase_cusum = 0.34422127985643525
phase_eta = 0.4185076471252436
phi21 = 1.1992706224488603
phi31 = 3.287013115332402
quartile31 = 0.34388388349331933
r21 = 0.10989838453044917
r31 = 0.07944797724428088
shapiro_w = 0.9642893075942993
skewness = 0.083168513620058
slope_per10 = -0.021631388803142193
slope_per90 = 0.023705766358141207
stetson_k = 0.8324125148637812
weighted_mean = 15.512864792724557
weighted_std = 0.17729615677855276


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
